In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import chardet
from unidecode import unidecode
from sqlalchemy import create_engine

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Lendo as variáveis de ambiente
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Criando a string de conexão
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Configurando a conexão com o PostgreSQL
engine = create_engine(connection_string)

In [2]:
# Definindo os tipos de dados
dtypes = {
    'cnpj_basico': 'str',  # Assumindo que o CNPJ Básico é numérico
    'opcao_pelo_simples': 'category', 
    'data_de_opcao_pelo_simples': 'str',  # Carregar como string inicialmente
    'data_de_exclusao_do_simples': 'str',  # Carregar como string inicialmente
    'opcao_pelo_mei': 'category',
    'data_de_opcao_pelo_mei': 'str',  # Carregar como string inicialmente
    'data_de_exclusao_do_mei': 'str'  # Carregar como string inicialmente
}


# Função para processar cada chunk
def process_chunk(chunk):
    chunk['data_de_opcao_pelo_simples'] = pd.to_datetime(chunk['data_de_opcao_pelo_simples'].str.strip(), format='%Y%m%d', errors='coerce')
    chunk['data_de_exclusao_do_simples'] = pd.to_datetime(chunk['data_de_exclusao_do_simples'].str.strip(), format='%Y%m%d', errors='coerce')
    chunk['data_de_opcao_pelo_mei'] = pd.to_datetime(chunk['data_de_opcao_pelo_mei'].str.strip(), format='%Y%m%d', errors='coerce')
    chunk['data_de_exclusao_do_mei'] = pd.to_datetime(chunk['data_de_exclusao_do_mei'].str.strip(), format='%Y%m%d', errors='coerce')
    
    # Aplicando a função unaccent em todas as colunas do DataFrame
    chunk = chunk.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)

    chunk.to_sql('simples', engine, if_exists='append', index=False)

# Lendo o CSV em chunks e processando cada chunk
chunksize = 100000  # Ajuste o tamanho do chunk conforme necessário
for chunk in pd.read_csv('../dados_csv/simples.csv', sep=';', names=list(dtypes.keys()), dtype=dtypes, chunksize=chunksize):
    process_chunk(chunk)

print("DataFrame enviado para o PostgreSQL com sucesso!")


/tmp/ipykernel_12755/3428674372.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chunk = chunk.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
/tmp/ipykernel_12755/3428674372.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chunk = chunk.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
/tmp/ipykernel_12755/3428674372.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chunk = chunk.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
/tmp/ipykernel_12755/3428674372.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chunk = chunk.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
/tmp/ipykernel_12755/3428674372.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chunk = chunk.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
/tmp/

DataFrame enviado para o PostgreSQL com sucesso!
